In [ ]:
from qu_los_sim import make_los
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
parameters = {
              "pi"  : [1.0],
              "phi" : [20.0],
              "psi" : [10.0],
              "sig" : [1.0],
              "dphi": [0.0]
            }
freqs = np.arange(400e6,2000e6,0.5e6)


In [ ]:
results = make_los(parameters, freqs=freqs, noise=0.01, do_rmsynth=False)
print(results.keys())


In [ ]:
plt.scatter(results['lsq'],results['pol'].real, s=1)
plt.scatter(results['lsq'],results['pol'].imag, s=1)
plt.scatter(results['lsq'],abs(results['pol']), s=1)


In [ ]:
plt.plot(results['fdf_dirty'][1]['phiArr_radm2'], abs(results['fdf_dirty'][1]['dirtyFDF']))
plt.xlim(-100,100)
